In [1]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [2]:
import pandas as pd
from urlparse import urlparse
import tldextract
from collections import Counter
import numpy as np
import json
import math
import re

In [3]:
from time import time
import math
import re

In [4]:
huizhou_stop_words = [u'的', u'了', u'在', u'是', u'安徽', u'和', u'有', u'我', u'年', u'上', u'他', u'我们', 
                      u'为', u'与', u'人', u'也', u'于', u'中', u'就', u'都', u'在线', u'等', u'不', u'网站', u'你', 
                      u'一个', u'到', u'之', u'对', u'说', u'进入', u'后', u'月', u'将', u'时', u'又', u'被', u'会员'
                     u'版权', u'论坛', u'以', u'而', u'着', u'下', u'这', u'但', u'要', u'或', u'首页', u'这',
                     u'地', u'从', u'卡', u'来', '多', u'日', u'她', u'还', u'联系', u'那', u'一处', u'一直', u'个', 
                     u'向', u'并', u'曾', u'这样', u'里', u'合肥', u'推荐', u'专栏', u'相关', u'当前', u'所有', u'中心'
                     u'频道', u'编辑', u'位置', u'第一', u'社区', u'看', '中安', u'安徽省', u'大', u'许可证', u'许可',
                     u'让', u'没有', u'什么', u'其', '请', '该', '用', '积分', u'凤阳', u'一篇', u'版权', u'县',
                     ]

In [5]:
class Toolbox (object):
    def __init__(self):
        self.init = 1
        
    def sort_dict (self, d, max=50):
        t = sorted(d.items(), key=lambda x: (-x[1], x[0]))
        return t[:max]
    
    def add_dict (self, x, y):
        d = { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }
        return d
    
    def create_dict(self, k1, v1, filters = True):
        if filters == True:
            k1_list0 = k1.strip('()').split(',')
            v1_list0 = v1.strip('()').split(',')
            #print v1_list0
            #print k1_list0
            k1_list1 = filter(None, k1_list0)        
            v1_list1 = filter(None, v1_list0)
        else:
            k1_list1 = k1
            v1_list1 = v1
        
        #print ' '.join(k1_list1)
        k2_list1 = [item.decode('utf-8') for item in k1_list1]
        v2_list1 = [float(item) for item in v1_list1]

    def create_vec(self, k1, v1):
        return (zip(k1, v1))
    
    def get_dict_top_n(self, d, max=50):
        ret = self.sort_dict(d, max)
        words, fq = zip(*ret)
        sum_up = sum(fq)
        fq_normalized = tuple(item/sum_up for item in fq)
        
        return dict(zip(words, fq_normalized))
    
    #calculate cosine between two dictionaries
    def get_sim(self, vec1, vec2):
        
        #get intersection
        intersection = set(vec1.keys()) & set(vec2.keys())
        
        #print intersection
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        #print 'num ---', numerator
        #print vec1.keys()
        #print vec1.values()
        #print vec2.keys()
        #print vec2.values()        
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
        
    def sum_vects(self, vects):
        sum_vects = None
        for vec in vects:
            if sum_vects == None:
                sum_vects = vec
            else:
                sum_vects = self.add_dict(sum_vects, vec)
        #only return first 100 after additions
        #print type(sum_vecs)
        return self.get_dict_top_n(sum_vects)
    
    def unserialized(self, x, eval_val=True):
        #print '----- in'
        #print x
        d = dict()
        s = x.strip('{}')
        #s = s.strip('[]')
        ret = s.split(',')

        #print ret
        for t in ret:
            r = t.split(':')
            #print r[0], r[1]
            key = eval(r[0].decode('utf-8'))
            if eval_val == True:
                d[key] = float(r[1])
            else:
                d[key] = r[1]
        #print len(ret)
        #print 'split -----'
        #print ' '.join(ret)
        return d
        
    def get_centroid_df(self, check, t_sim, num=2000):        
        if len(check) > num:
            #sampling     
            ratio = num*1.0/len(check)
            print 'ratio ---', ratio 
            msk = np.random.rand(len(check)) < ratio
            tmp = check[msk]
            check = tmp
    
        check.info()
        check_vecs = check['s_vec']
        return self.sum_vects(check_vecs) 
    
    def set_contained_flag(self, x, y):
        x_keys = x.keys()
        y_keys = y.keys()
        for s in y_keys:
            if s in x_keys:
                return True
        return False

In [8]:
aT = Toolbox()

In [61]:
df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-slim_v1.1.csv')

In [ ]:
df['url'] 

In [62]:
#unseralized
t0 = time()
df['s_vec'] = df.s_vec.map(lambda x: aT.unserialized(x))
print time() - t0 

In [162]:
df.info()

In [64]:
def map_func(x):
    s = x.split(',')
    t = [r.split(':')[0] for r in s]
    print ' '.join(t)

In [65]:
#calculte similarity
def calculate_sim(x, in_dict):
    #print in_dict
    tb = Toolbox()
    return tb.get_sim(x, in_dict)

In [66]:
#merge the dict
def map_merge_vec(k, v):
    tb = Toolbox()
    return tb.create_dict(k, v)

In [67]:
#Get contents for '徽州'
s_dict = dict()
s_dict[u'徽文化'] = 0.5
s_dict[u'徽州'] = 0.5
t_sim = 'hz_sim'

In [68]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [69]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]
idx = 0

In [70]:
#print out centroid 
vec1 = centroid_vec
ret = aT.sort_dict(centroid_vec)
words, cnt = zip(*ret)
print 'centroid --- for 徽文化'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [71]:
#Get contents for '徽学'
s_dict = dict()
s_dict[u'徽学'] = 0.4
s_dict[u'文书'] = 0.3
s_dict[u'签约'] = 0.3
t_sim = 'hx_sim'


In [72]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [73]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [74]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec2 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 徽学'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [75]:
idx = 0
for i in s_sim:
    if out_df.iloc[i][t_sim] > 0.3:
        title = out_df.iloc[i]['title']
        sim_val = out_df.iloc[i][t_sim]
        idx += 1
        print idx, i, title, sim_val

In [76]:
#Get contents for '新安'
s_dict = dict()
s_dict[u'新安'] = 0.4
s_dict[u'理学'] = 0.3
s_dict[u'风水'] = 0.3
s_dict
t_sim = 'xa_sim'

In [77]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [78]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [79]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec3 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 新安'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [80]:
idx = 0
for i in s_sim:
    if out_df.iloc[i][t_sim] > 0.3:
        title = out_df.iloc[i]['title']
        sim_val = out_df.iloc[i][t_sim]
        idx += 1
        print idx, i, title, sim_val

In [81]:
#Get contents for '古村落'
s_dict = dict()
s_dict[u'古村落'] = 0.2
s_dict[u'民居'] = 0.2
s_dict[u'祠堂'] = 0.2
s_dict[u'旅游'] = 0.2
s_dict[u'徽州'] = 0.2

s_dict
t_sim = 'cl_sim'

In [82]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [83]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [84]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec4 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 古村落'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [85]:
idx = 0
for i in s_sim:
    if out_df.iloc[i][t_sim] > 0.3:
        title = out_df.iloc[i]['title']
        sim_val = out_df.iloc[i][t_sim]
        url = out_df.iloc[i]['url']
        
        idx += 1
        print idx, i, title, sim_val
        print url

In [86]:
#Get contents for '徽雕'
s_dict = dict()
s_dict[u'徽雕'] = 0.3
s_dict[u'雕刻'] = 0.2
s_dict[u'三雕'] = 0.2
s_dict[u'建筑'] = 0.2

t_sim = 'hd_sim'

In [87]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [88]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [89]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec5 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 徽雕'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [90]:
idx = 0
for i in s_sim:
    if out_df.iloc[i][t_sim] > 0.3:
        title = out_df.iloc[i]['title']
        sim_val = out_df.iloc[i][t_sim]
        idx += 1
        print idx, i, title, sim_val

In [91]:
#Get contents for '徽商'
s_dict = dict()
s_dict[u'徽商'] = 0.4
s_dict[u'罗盘'] = 0.3
s_dict[u'茶叶'] = 0.3

t_sim = 'hs_sim'

In [92]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [93]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [94]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec6 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 徽商'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [95]:
idx = 0
for i in s_sim:
    if out_df.iloc[i][t_sim] > 0.3:
        title = out_df.iloc[i]['title']
        sim_val = out_df.iloc[i][t_sim]
        idx += 1
        print idx, i, title, sim_val

In [96]:
#Get contents for '徽剧'
s_dict = dict()
s_dict[u'徽剧'] = 0.4
s_dict[u'徽墨'] = 0.2
s_dict[u'徽砚'] = 0.2
s_dict[u'徽菜'] = 0.2

t_sim = 'hj_sim'

In [97]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [98]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [99]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec7 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 徽剧'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [100]:
#Get contents for '民俗'
s_dict = dict()
s_dict[u'民俗'] = 0.2
s_dict[u'教育'] = 0.2
s_dict[u'历史'] = 0.2
s_dict[u'博物馆'] = 0.2
s_dict[u'徽州'] = 0.2

t_sim = 'ms_sim'

In [101]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [102]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [103]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec8 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 民俗'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [104]:
#Get contents for '历史'
s_dict = dict()
s_dict[u'名人'] = 0.3
s_dict[u'人物'] = 0.3
s_dict[u'徽州'] = 0.4

t_sim = 'ls_sim'

In [105]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [106]:
out_df = df[df[t_sim] > 0.50]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [107]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec9 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 人物'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [108]:
#Get contents for '旅游'
s_dict = dict()
s_dict[u'徽州'] = 0.4
s_dict[u'旅游'] = 0.3
s_dict[u'推荐'] = 0.3

s_dict
t_sim = 'ly_sim'

In [109]:
t0 = time()

#df[t_sim] = df.s_vec.map(lambda x: calculate_sim(x, dsi_dict))
df['s_check'] = df.s_vec.map(lambda x: aT.set_contained_flag(x, s_dict))
check = df[df['s_check'] == True]
print 'after s_check', time() - t0

print '---- check len---', len(check)

#cc = check[:10]

centroid_vec = aT.get_centroid_df(check, t_sim)
print 'after centroid', time() - t0
#only work on non duplicated ones
df[t_sim] = df.s_vec.map(lambda x: aT.get_sim(x, centroid_vec))
s_sim = df[t_sim].values.argsort()[::-1]
print time() - t0

In [110]:
out_df = df[df[t_sim] > 0.55]
out_df.info()
s_sim = out_df[t_sim].values.argsort()[::-1]

In [111]:
#print out centroid 
ret = aT.sort_dict(centroid_vec)
vec10 = centroid_vec

words, cnt = zip(*ret)
print 'centroid --- for 人物'
for i in range(0, len(words)):
    print i, words[i], cnt[i]

In [112]:
idx = 0
s_sim = df['hz_sim'].values.argsort()[::-1]

for i in s_sim:
    title = df.iloc[i]['title']
    sim_val = df.iloc[i]['hz_sim']
    print(idx, i, sim_val, title)
    idx += 1

In [113]:
aT.get_sim(vec1, vec8)

0.9554316768725106

In [114]:
df.info()

In [180]:
ratio = 0.42
#title, 'hz_sim', 'hx_sim', 'xa_sim', 'cl_sim', 'hd_sim', 'hs_sim', 'ms_sim', 'ls_sim', 'ly_sim' 
new_df = df[(df['hz_sim'] > ratio) | (df['hd_sim'] > ratio) | (df['ls_sim'] > ratio) | 
            (df['cl_sim'] > ratio) | (df['hj_sim'] > ratio) | (df['ms_sim'] > ratio) |  
            (df['hs_sim'] > ratio) | (df['xa_sim'] > ratio) | (df['hx_sim'] > ratio)]

In [209]:
new_df.info()

In [170]:
new_df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-slim_r_0_5.csv', encoding='utf-8', index=False)

In [765]:
new_df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-slim_v1.2.csv')

In [766]:
#Clean up for web app

In [182]:
def get_summary(t):
    
    if u'------分隔线----------------------------' in t:
        print 'found one -------'
    x = t.replace(u'------分隔线----------------------------', '')    
    x = x.replace(u'————————————————————————————————————————————', '')
    x = x.replace('！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！。！！！！！！！！！！！！！！！！！！', '')
    sentences = x.split('\n')
    #print 'len -----', len(sentences)
    #skip the first sentences
    if len(sentences) > 3:
        s = '。'.join(sentences[1:]).decode('utf-8')
    else:
        s = x.strip().decode('utf-8')
    if len(s) > 100:
        ret = ''.join(s[:100]) + '...'
    else:
        ret = s
    
    if len(s) < 10:
        ret = x
        
    if ret[0] == u'。':
        return ret[1:].strip()
    else:
         return ret.strip()

In [183]:
t = ['test', 'test2']
print '。'.join(t) 

In [184]:
global idx
idx = 0
def map_strip_titles(x):
    global idx
    #print idx
    #print x
    x = x.replace('-故园徽州网 - Powered by Discuz!', '')
    x = x.replace('婺源论坛 - 江西婺源旅游 - 婺源老家旅游论坛第一门户', '')
    x = x.replace('-中国华文教育网', '')
    x = x.replace('-中安在线-徽文化', '')
    x = x.replace('中国华文教育网－', '')

    
    t = x.split('_')
    s = t[0]
    idx += 1
    #print s
    return s
    

In [121]:
orig_df = pd.read_csv('/Users/nhu2000/projects/SimilarSearch/data/final/huizhou-final-v1.csv')

In [151]:
orig_df['url'] = orig_df.url.map(lambda x: map_update_url(x))

In [152]:
orig_df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/final/huizhou-final-v2.csv', encoding='utf-8', index=False)

In [122]:
orig_df.info()

In [185]:
def get_content_by_url(url):
    df = orig_df[orig_df['url'] == url]
    return df.iloc[0]['content']

In [186]:
import unicodedata
def remove_non_chinese(s):
    r = [c for c in s if unicodedata.category(c).startswith('L')]
#print r
    return ''.join(r)

In [125]:
import re
def map_update_summary(x, y):
    s = remove_non_chinese(x.decode('utf-8'))
    if re.match('^[0-9a-z]+$',s.lower()):
        print 'length =', len(s)        
        return y
    else:
        return x

In [142]:
def map_update_url(x):
    if 'http' not in x:
        return 'http://' + x
    else:
        return x

In [187]:
new_df['url'] = new_df.url.map(lambda x: map_update_url(x))

In [188]:
new_df['title'] = new_df.title.map(lambda x: map_strip_titles(x))

In [189]:
new_df['content'] = new_df.url.map(lambda x: get_content_by_url(x))

In [190]:
new_df['summary'] = new_df.content.map(lambda x: get_summary(x))

In [191]:
new_df['summary'] = (map(map_update_summary, new_df['summary'],new_df['title']))

new_df['s_vec_str'] = new_df.s_vec.map(lambda x: str(x))
out_new_df = new_df.dropna(subset = ['s_vec_str'])

In [201]:
new_df.info()

In [192]:
new_df.info()

In [ ]:
new_df.head()

In [193]:
for i in range(0, len(new_df)):
    url = new_df.iloc[i]['url']
    t = new_df.iloc[i]['title']
    s = new_df.iloc[i]['summary']
    #print i, t
    #print s
    if 'http:' not in url: #i == 6255:
        print url
        print t

In [202]:
new_df.info()

In [194]:
out_df = new_df[['url', 'title', 'summary', 's_vec', 'hz_sim', 'hd_sim', 'ls_sim', 'cl_sim', 'hj_sim', 
                'ms_sim', 'hs_sim', 'xa_sim', 'hx_sim']]

In [195]:
out_df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-slim_v1_3.csv', encoding='utf-8', index=False)

In [210]:
out_df_full.info()

In [203]:
out_df_full = new_df[['url', 'title', 's_vec',  'content']]

In [197]:
def get_content_seg_by_url(url):
    df = orig_df[orig_df['url'] == url]
    return df.iloc[0]['content_seg']

In [198]:
out_df_full['content_seg'] = out_df_full.url.map(lambda x: get_content_seg_by_url(x))

In [204]:
out_df_full.head()

,url,title,s_vec,content
5,http://cul.anhuinews.com/system/2016/01/26/007...,徽州古民居雕刻装饰研究,"{u'是从': 0.0189873417722, u'还有': 0.012658227848...",徽州因明清两朝的繁盛而在历史上留下浓墨重彩的一笔，建筑就是那个时代的重要表现之一。当时的徽商...
7,http://cul.anhuinews.com/system/2014/07/11/006...,徽州为什么称为“文献之邦”,"{u'宋代': 0.0142857142857, u'孤本': 0.014285714285...",徽州作为极具特色的地域文化区，在宋代就已经基本形成。明代以来徽商迅速崛起，更加有力的推动了教...
9,http://cul.anhuinews.com/system/2014/06/26/006...,安徽省非物质文化遗产：砖雕、石雕、木雕,"{u'迄今': 0.012987012987, u'雕刻': 0.038961038961,...","古代徽州辖地包括今天的安徽省黄山市和江西省婺源县。""徽州三雕""为古代徽州地区明清建筑的装饰性..."
10,http://cul.anhuinews.com/system/2014/06/26/006...,徽州文化中的烟文化,"{u'地方': 0.01, u'理应': 0.01, u'哪个': 0.01, u'加工':...",徽州文化的内容包罗万象，如徽州土地制度、徽商、徽州历史名人、徽州科技、新安理学、新安医学、徽...
12,http://cul.anhuinews.com/system/2014/06/19/006...,马头墙内外的徽州女人,"{u'信条': 0.00909090909091, u'贞节牌坊': 0.018181818...",安徽南端的徽州地域，黄山白岳新安江山水秀美，在蓝天白云绿水青山掩映之中，徽州古村落高低起伏、...


In [205]:
out_df_full.to_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou-web-full_v1_3.csv', encoding='utf-8', index=False)

In [211]:
df.info()

In [212]:
df.to_csv('/Users/nhu2000/projects/SimilarSearch/data/clean/huizhou_full_v2_0.csv', encoding='utf-8', index=False)